In [1]:
from fuzzy_ngram_matrix_factors_model import SentimentModel

In [2]:
heavy_reg_model_v1 = SentimentModel.load_from_checkpoint('lightning_logs/heavy_regularization/version_1/checkpoints/last.ckpt', no_load_glove=False)
heavy_reg_model_v1.eval()

Loading GloVe... Done!


SentimentModel(
  (embedding): Embedding(1500000, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
    (3): Conv2d(1, 100, kernel_size=(7, 300), stride=(1, 1))
  )
  (user_embedding): Embedding(123356, 10)
  (product_embedding): Embedding(49945, 10)
  (user_product_dim_reduction): Identity()
  (user_product_pool): Identity()
  (feature_weights): Linear(in_features=822, out_features=5, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [3]:
from preprocess_data import load_data

train_df, test_df = load_data()

Loading preprocessed Parquet files...... Done!


In [4]:
train_df.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Summary_glove_tokens,Text_glove_tokens,Summary_glove_tokens_np,Text_glove_tokens_np,HelpfulnessRatio,LogHelpfulnessDenominator
0,914403,B0009W5KHM,AV6QDP8Q0ONK4,2,2,1341014400,GOOD FUN FILM,While most straight to DVD films are not worth...,5.0,"[112, 487, 667]","[212, 122, 1681, 4, 6131, 2802, 23, 35, 1016, ...","[112, 487, 667]","[212, 122, 1681, 4, 6131, 2802, 23, 35, 1016, ...",1.000000,1.098612
1,354887,6303079709,A2I8RXJN80A2D2,0,0,1168819200,Movie Review,"I have wanted this one for sometime, also. I ...",5.0,"[603, 555]","[108, 31, 652, 27, 51, 11, 0, 292099, 108, 777...","[603, 555]","[108, 31, 652, 27, 51, 11, 0, 292099, 108, 777...",0.000000,0.000000
2,1407653,B004H0M2XC,A3FHV3RV8Z12E6,0,0,1386201600,When is it a good time to Consent?,Actually this was a pretty darn good indie fil...,4.0,"[82, 10, 21, 6, 112, 77, 4, 0]","[406, 27, 30, 6, 491, 15552, 112, 12431, 0, 21...","[82, 10, 21, 6, 112, 77, 4, 0]","[406, 27, 30, 6, 491, 15552, 112, 12431, 0, 21...",0.000000,0.000000
3,1377458,B003ZJ9536,A12VLTA3ZHVPUY,1,1,1348704000,TRUTH,Episodes 37 to 72 of the series press on in a ...,5.0,[1748],"[5579, 2983, 4, 3652, 5, 2, 633, 1599, 17, 7, ...",[1748],"[5579, 2983, 4, 3652, 5, 2, 633, 1599, 17, 7, ...",1.000000,0.693147
4,475323,630574453X,A13NM1PES9OXVN,2,3,970012800,Intelligent and bittersweet -- stays with you,"I was really impressed with this movie, but wa...",3.0,"[6031, 3, 33699, 165, 7048, 19, 18]","[108, 30, 155, 4734, 19, 27, 0, 42, 0, 6, 308,...","[6031, 3, 33699, 165, 7048, 19, 18]","[108, 30, 155, 4734, 19, 27, 0, 42, 0, 6, 308,...",0.666667,1.386294


In [7]:
import torch
from model_utils import (
    initialize_environment, load_trained_model, process_data, create_dataloader, generate_submission, test_model_on_samples
)
from data_processor import DataProcessor
from preprocess_data import load_data
from utils import seed_everything

device = initialize_environment(seed=3)


Seed set to 3


In [10]:

# Load encoders
data_processor = DataProcessor()
data_processor.load_encoders()

# Sample a very small subset of the training data
train_df_sample = train_df.sample(n=10, random_state=3)  # You can adjust 'n' for a larger or smaller subset

# Process the small subset (to avoid preprocessing the entire train_df)
processed_sample = process_data(train_df_sample, data_processor, is_training=False)

# Load the trained model
heavy_reg_model_v1 = SentimentModel.load_from_checkpoint(
    'lightning_logs/heavy_regularization/version_1/checkpoints/last.ckpt', no_load_glove=False
)
heavy_reg_model_v1.eval()

# Test the model on this small sample
test_model_on_samples(
    model=heavy_reg_model_v1, 
    data_df=processed_sample, 
    data_processor=data_processor, 
    device=device, 
    num_samples=1
)


Sample 1:
Review ID: 817514
User ID: A2L74BOWNZU374
Product ID: B0001Z3IK8
Text: Run Angel Run:  1 starHell's Bloody Devils:  1 1/2 starsHell's Angels 69:  3 starsThe only movie worth watching of the three in this set is Hells Angels 69, the other two are too lame even to review nevermind watch. First of all the rumble that rocked Las Vegas never ocurred in this film which did lack action. Still it was well made and it features many original members of the hells angels Oakland chapter. It is an ok biker flick. Born Losers and Beyond the Law are worth checking out if you're looking for a good biker movie. Satans Sadists is probably the best biker flick I've seen.
Summary: Nothing great at all here, most disappointing in fact
Prediction: 3
Ground Truth: 2.0
--------------------------------------------------


In [9]:
processed_sample

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Summary_glove_tokens,Text_glove_tokens,Summary_glove_tokens_np,Text_glove_tokens_np,HelpfulnessRatio,LogHelpfulnessDenominator,user_idx,product_idx
960978,315464,6302484421,A3ESD5FA6HNLUE,3,3,1062374400,Neat little thriller,A movie that you can watch repetively and enjo...,5.0,"[8413, 211, 15042]","[6, 603, 15, 18, 41, 712, 0, 3, 777, 7426, 146...","[8413, 211, 15042]","[6, 603, 15, 18, 41, 712, 0, 3, 777, 7426, 146...",1.000000,1.386294,78606,5577
164037,713859,B00006FI0R,A3GAEVX42E80I1,1,1,1025481600,Jason X,I'm a person that usually doesn't explain the ...,3.0,"[29798, 359]","[0, 6, 377, 15, 821, 0, 2443, 2, 603, 266, 6, ...","[29798, 359]","[0, 6, 377, 15, 821, 0, 2443, 2, 603, 266, 6, ...",1.000000,0.693147,79960,15986
532260,689299,B00005Y70Q,ASYCE9334ANVQ,5,5,1219795200,Late era film noir in color,A reform candidate's efforts to clean up a mob...,4.0,"[964, 4412, 667, 30566, 7, 866]","[6, 5043, 0, 2017, 4, 1186, 61, 6, 0, 1031, 23...","[964, 4412, 667, 30566, 7, 866]","[6, 5043, 0, 2017, 4, 1186, 61, 6, 0, 1031, 23...",1.000000,1.791759,116974,15068
381341,817514,B0001Z3IK8,A2L74BOWNZU374,0,0,1283126400,"Nothing great at all here, most disappointing ...",Run Angel Run: 1 starHell's Bloody Devils: 1...,2.0,"[578, 158, 25, 43, 0, 122, 11504, 7, 467]","[459, 7473, 0, 66, 0, 9418, 0, 66, 1979, 0, 10...","[578, 158, 25, 43, 0, 122, 11504, 7, 467]","[459, 7473, 0, 66, 0, 9418, 0, 66, 1979, 0, 10...",0.000000,0.000000,52030,20305
920021,829980,B0002C9D9K,A8N5EM32PVLZJ,3,5,1269302400,"A Man of His Time, and the Ages",Would we die for our faith? Many of us hope th...,4.0,"[6, 309, 5, 60, 0, 3, 2, 3556]","[69, 48, 2086, 11, 70, 0, 142, 5, 140, 534, 15...","[6, 309, 5, 60, 0, 3, 2, 3556]","[69, 48, 2086, 11, 70, 0, 142, 5, 140, 534, 15...",0.600000,1.791759,98619,20812
225808,655063,B00005JPOE,A3H075I91939S0,0,0,1355875200,GOOD CONDITION & FAST SERVICE!,Good Acting but I had a hard time following th...,4.0,"[112, 1656, 63, 1014, 0]","[112, 3779, 42, 108, 81, 6, 375, 77, 413, 2, 2...","[112, 1656, 63, 1014, 0]","[112, 3779, 42, 108, 81, 6, 375, 77, 413, 2, 2...",0.000000,0.000000,80602,13784
509237,1276802,B002LMSWN2,ALUCONU0OJJ0S,5,10,1256083200,One that gets it right,"Believe the hype, this is a terrific movie. T...",4.0,"[51, 15, 759, 21, 169]","[488, 2, 0, 27, 10, 6, 9023, 252083, 2, 287, 1...","[51, 15, 759, 21, 169]","[488, 2, 0, 27, 10, 6, 9023, 252083, 2, 287, 1...",0.500000,2.397895,110650,38204
207605,1053133,B000N3T0DW,AT07UZQQR7ZEH,2,4,1177027200,A fun Spanish dark comedy with strong women,Pedro Almodovar is Spain's greatest contempora...,4.0,"[6, 487, 14107, 1640, 4972, 19, 957, 439]","[94392, 1119772, 10, 0, 2671, 3921, 0, 60, 143...","[6, 487, 14107, 1640, 4972, 19, 957, 439]","[94392, 1119772, 10, 0, 2671, 3921, 0, 60, 143...",0.500000,1.609438,117022,29889
1177737,635408,B00005JNTI,A3IMNZSYDOTTU6,3,9,1132272000,A Great Sci-Fi Masterpiece...Spielberg and Cru...,I remember earlier this year eagerly awaiting ...,5.0,"[6, 158, 19243, 0, 3, 5561, 0]","[108, 849, 1658, 27, 152, 18944, 13478, 2, 102...","[6, 158, 19243, 0, 3, 5561, 0]","[108, 849, 1658, 27, 152, 18944, 13478, 2, 102...",0.333333,2.302585,82070,13638
886718,10772,0767811100,A1FI9Y1OX5K5PE,3,4,1153785600,So funny you can't stop laughing,It's no wonder this movie won so many Oscars! ...,5.0,"[59, 1558, 18, 1184154, 709, 6556]","[727538, 96, 1817, 27, 603, 1332, 59, 142, 0, ...","[59, 1558, 18, 1184154, 709, 6556]","[727538, 96, 1817, 27, 603, 1332, 59, 142, 0, ...",0.750000,1.609438,14044,173
